Title
-----------------------------------------

Description

Copyright 2016 Allen Downey

MIT License: http://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

import thinkbayes2
import thinkplot

from thinkbayes2 import Pmf, Suite

import numpy as np
from scipy import stats

%matplotlib inline

Suppose we test a patient to see if they have a disease, and the test comes back positive.  What is the probability that the patient is actually sick (that is, has the disease)?

To answer this question, we need to know:

*  The prevalence of the disease in the population the patient is from.  Let's assume the patient is identified as a member of a population where the prevalence is `p`.

*  The sensitivity of the test, `s`, which is the probability of a positive test if the patient is sick.

*  The false positive rate of the test, `t`, which is the probability of a positive test if the patient is not sick.

Given these parameters, we can compute the probability that the patient is sick, given a positive test.

### Test class

To do that, I'll define a `Test` class that extends `Suite`, so it inherits `Update` and provides `Likelihood`.

The instance variables of `Test` are:

*  `p`, `s`, and `t`: Copies of the parameters.
*  `d`: a dictionary that maps from hypotheses to their probabilities.  The hypotheses are the strings `sick` and `notsick`.
*  `likelihood`: a dictionary that encodes the likelihood of the possible data values `pos` and `neg` under the hypotheses.



In [2]:
class Test(Suite):
    def __init__(self, p, s, t):
        self.p = p
        self.s = s
        self.t = t
        d = dict(sick=p, notsick=1-p)
        super(Test, self).__init__(d)
        self.likelihood = dict(pos=dict(sick=s, notsick=t),
                               neg=dict(sick=1-s, notsick=1-t))
        
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: 'sick' or 'notsick'
        """
        return self.likelihood[data][hypo]

Now we can create a `Test` object with parameters chosen for demonstration purposes (most medical tests are better than this!):

In [3]:
p = 0.1      # prevalence
s = 0.9      # sensitivity
t = 0.3      # false positive rate
test = Test(p, s, t)
test.Print()

notsick 0.9
sick 0.1


And here's how we update the `Test` object with a positive outcome:

In [4]:
test.Update('pos')
test.Print()

notsick 0.75
sick 0.25


The positive test provides evidence that the patient is sick, increasing the probability from 0.1 to 0.25.

### Hierarchical model

So far, this is basic Bayesian inference.  Now let's add a wrinkle.  Suppose that we don't know the value of `t` with certainty, but we have reason to believe that `t` is either 0.2 or 0.4 with equal probability.

We can represent this belief with a hierarchical model, where the levels of the hierarchy are:

*  At the top level, the distribution of possible values for `t`, and their probabilities.
*  At the bottom level, the probability that the patient is sick or not, conditioned on `t`.

To represent the hierarchy, I'll define a `MetaTest`, which is a `Suite` that contains `Test` objects with different values of `t` as hypotheses.

In [5]:
class MetaTest(Suite):
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: Test object
        """
        return hypo.Update(data)
    
    def Print(self):
        for test, prob in self.Items():
            print('t=', test.t, test, prob)

To update a `MetaTest`, we update each of the hypothetical `Test` objects.  The return value from `Update` is the normalizing constant, which is the total probability of the data under the hypothesis.

We use the normalizing constants from the bottom level of the hierarchy as the likelihoods at the top level.

Here's how we create the `MetaTest` for the scenario we described:

In [6]:
q = 0.5
t1 = 0.2
t2 = 0.4

test1 = Test(p, s, t1)
test2 = Test(p, s, t2)

metatest = MetaTest({test1:q, test2:1-q})
metatest.Print()

t= 0.2 Test({'sick': 0.1, 'notsick': 0.9}) 0.5
t= 0.4 Test({'sick': 0.1, 'notsick': 0.9}) 0.5


At the top level, there are two tests, with different values of `t`.  Initially, they are equally likely.

When we update the `MetaTest`, it updates the embedded `Test` objects and then the `MetaTest` itself.

In [7]:
metatest.Update('pos')

0.36000000000000004

Here are the results.

In [8]:
metatest.Print()

t= 0.2 Test({'sick': 0.3333333333333333, 'notsick': 0.6666666666666666}) 0.375
t= 0.4 Test({'sick': 0.19999999999999998, 'notsick': 0.7999999999999999}) 0.625


Because a positive test is more likely if `t=0.4`, the positive test is evidence in favor of the hypothesis that `t=0.4`.

This `MetaTest` object represents what we should believe about `t` after seeing the test, and what we should believe about the probability that the patient is sick.

### Marginal probabilities

To compute the probability that the patient is sick, we have to compute the marginal probabilities of `sick` and `notsick`, averaging over the possible values of `t`.  The following function computes this marginal distribution:

In [9]:
def Marginal(metatest):
    pmf = thinkbayes2.Pmf()
    for test, p1 in metatest.Items():
        for hypo, p2 in test.Items():
            pmf[hypo] += p1 * p2
    return pmf

Here's the marginal posterior:

In [10]:
posterior = Marginal(metatest)
posterior.Print()

notsick 0.75
sick 0.25


After seeing the test, the probability that the patient is sick is 0.25, which is the same result we got with `t=0.3`.  So at first glance it seems like we might not need the hierarchical model at all.  Maybe we could have computed the mean of `t` and done a simple update.

### Two patients

If the goal is to make a single prediction for a single patient, that's true.  But let's try a different scenario.  Suppose you test two patients and they both test positive.  What is the probability that they are both sick?

To answer that, I'm going to define a `Patient` class that contains a `MetaTest` and provides some methods for interpreting the results:

In [11]:
class Patient:
    def __init__(self, p, s, pmf_t):
        tests = {}
        for t, q in pmf_t.Items():
            tests[Test(p, s, t)] = q
        self.metatest = MetaTest(tests)
        
    def Update(self, data):
        self.metatest.Update(data)
        
    def Marginal(self):
        return Marginal(self.metatest)
    
    def Conditional(self, t):
        for test, prob in self.metatest.Items():
            if test.t == t:
                return test, prob

`Marginal` computes the marginal probabilities of `sick` and `notsick`, averaging over the possible values of `t`.

`Conditional` takes a specified value for `t` and returns the probabilities of `sick` and `notsick` conditioned on `t`.

I'll test it out using the same parameters from above:

In [12]:
q = 0.5
t1 = 0.2
t2 = 0.4
pmf_t = Pmf({t1:q, t2:1-q})
pmf_t.Print()

0.2 0.5
0.4 0.5


Here are the results

In [13]:
patient1 = Patient(p, s, pmf_t)
patient1.Update('pos')
patient1.metatest.Print()

t= 0.2 Test({'sick': 0.3333333333333333, 'notsick': 0.6666666666666666}) 0.375
t= 0.4 Test({'sick': 0.19999999999999998, 'notsick': 0.7999999999999999}) 0.625


These are the same as the results we saw before.  Now we can extract the marginal distribution:

In [14]:
patient1.Marginal().Print()

notsick 0.75
sick 0.25


Again, same as what we saw before.  And we can extract the conditional distributions for `t1` and `t2`.

In [15]:
cond1, prob = patient1.Conditional(t1)
cond1.Print()

notsick 0.666666666667
sick 0.333333333333


In [16]:
cond2, prob = patient1.Conditional(t2)
cond2.Print()

notsick 0.8
sick 0.2


Now we're ready to answer questions about multiple patients.  Let's create another patient with a positive test:

In [17]:
patient2 = Patient(p, s, pmf_t)
patient2.Update('pos')
patient2.metatest.Print()

t= 0.2 Test({'sick': 0.3333333333333333, 'notsick': 0.6666666666666666}) 0.375
t= 0.4 Test({'sick': 0.19999999999999998, 'notsick': 0.7999999999999999}) 0.625


Now, to compute the probability that both patients are sick, we have to know the distribution of `t` for both patients.  And that depends on details of the scenario.  There are two possibilities:

*  Scenario A: Suppose there are two different tests, with different false positive rates, and we don't know which tests were used.  Or suppose that the false positive rate is different for different groups of people, and we don't know which group the patients are from.  In either case, the values of `t` for the two patients are independent.

*  Scenario B: Alternatively, suppose there is only one test and we have reason to believe that the false positive rate is the same for everyone, so `t` is either `t1` for everyone or `t2` for everyone, but we don't know which.  In that case, the value of `t` for the two patients is the same.

The probability that both patients are sick is different in these two scenarios.  I'll compute both.

### Scenario A

In Scenario A, we can compute the probability of `sick` and `nonsick` independently for the two patients and then compute the "sum" of those distributions:

In [18]:
conjunction = patient1.Marginal() + patient2.Marginal()
conjunction.Print()

notsicknotsick 0.5625
notsicksick 0.1875
sicknotsick 0.1875
sicksick 0.0625


In Scenario A, the probability that both patients are sick is 0.0625, which is just the square of the probability that each patient is sick, 0.25.

### Scenario B

In Scenario B we have to work a little harder.  For a given value of `t` we can compute the probability of `sicksick` by conjoining the conditional distributions for the two patients:

In [19]:
cond1, p_t1 = patient1.Conditional(t1)
cond2, p_t1 = patient2.Conditional(t1)
conjunction_t1 = cond1 + cond2
conjunction_t1.Print()

notsicknotsick 0.444444444444
notsicksick 0.222222222222
sicknotsick 0.222222222222
sicksick 0.111111111111


If we know that `t=t1`, the probability of `sicksick` is 0.111.  And for `t=t2`:

In [20]:
cond1, p_t2 = patient1.Conditional(t2)
cond2, p_t2 = patient2.Conditional(t2)
conjunction_t2 = cond1 + cond2
conjunction_t2.Print()

notsicknotsick 0.64
notsicksick 0.16
sicknotsick 0.16
sicksick 0.04


If we know that `t=t2`, the probability of `sicksick` is 0.04.

The overall probability of `sicksick` is the weighted average of these probabilities:

In [21]:
p_t1 * conjunction_t1['sicksick'] + p_t2 * conjunction_t2['sicksick']

0.06666666666666665

We can generalize that process to compute all of the posterior probabilities.

The following function computes the conjunctions:

In [22]:
def mysum(seq):
    return sum(seq[1:], seq[1])

def MakeConjunction(patients, t):
    t = [patient.Conditional(t) for patient in patients]
    conds, probs = zip(*t)
    return mysum(conds), probs[0]

Here's an example that computes the conjunction for `t=t1`:

In [23]:
ts = pmf_t.Values()
patients = [patient1, patient2]

pmf, prob = MakeConjunction(patients, t1)
print(prob)
pmf.Print()

0.375
notsicknotsick 0.444444444444
notsicksick 0.222222222222
sicknotsick 0.222222222222
sicksick 0.111111111111


Now we can make a MetaTest that contains the conjunctions with the right weights:

In [24]:
metapmf = Pmf()
for t in ts:
    pmf, prob = MakeConjunction(patients, t)
    metapmf[pmf] = prob
    
metapmf.Print()

Pmf({'sicksick': 0.039999999999999994, 'sicknotsick': 0.15999999999999998, 'notsicknotsick': 0.6399999999999999, 'notsicksick': 0.15999999999999998}) 0.625
Pmf({'sicksick': 0.1111111111111111, 'sicknotsick': 0.2222222222222222, 'notsicknotsick': 0.4444444444444444, 'notsicksick': 0.2222222222222222}) 0.375


And finally we can use `Marginal` to compute the weighted averages of the posterior probabilities:

In [25]:
Marginal(metapmf).Print()

notsicknotsick 0.566666666667
notsicksick 0.183333333333
sicknotsick 0.183333333333
sicksick 0.0666666666667


### Summary

In summary:

*  In Scenario A, the values of `t` for the two patients are independent, and the probability that both are sick is 0.0625.

*  In Scenario B, the value of `t` has to be the same for both patients, and the probability that both are sick is 0.0666.

A real scenario might combine elements of both; that is, the false positive rate might be different for different people, and we might have some uncertainty about what it is.  In that case, the most accurate predictive probability might be anywhere between the values we computed.